Check Tensorflow Version

In [ ]:
import tensorflow as tf
tf.__version__

Connect to Google Drive
The filled folders boat and non_boat with the C++ code reported in 'Training' have been uploaded 
Move to the folder that contains those direcories

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Loading in the packages




In [ ]:
import numpy as np
import argparse
import pickle
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os,tensorflow.keras
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.metrics import classification_report
from tensorflow.keras import Model
from tensorflow.keras import optimizers

Create two empty list that will be filled from the two directory of Google Drive

In [ ]:
images = [];
labels = [];

test_images = [];
test_labels = [];

In [ ]:
import glob
import os
import re

boat_filenmaes = glob.glob(r"/----path to the folder ----/boat/*.jpg")
non_boat_filenames = glob.glob(r"/----path to the folder ----/non_boat/*.jpg")
boat_test = glob.glob(r"/----path to the folder ----/boat_test/*.jpg")
non_boat_test = glob.glob(r"/----path to the folder ----/non_boat_test/*.jpg")

for i, file_path in enumerate(boat_filenames):
    print(i)
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    labels.append([1])
    
for i, file_path in enumerate(non_boat_filenames):
    print(i)
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    labels.append([0])

for i, file_path in enumerate(boat_test):
    print(i)
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    test_images.append(img)
    test_labels.append([1])

for i, file_path in enumerate(non_boat_test):
    print(i)
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    test_images.append(img)
    test_labels.append([0])

In [ ]:
# conversion from list to np.array, this is the training dataset
X_train = np.array(images)
Y_train = np.array(labels)

print(X_train.shape)
print(Y_train.shape)

In [ ]:
# conversion from list to np.array, this is the test part
X_test = np.array(test_images)
Y_test = np.array(test_labels)

print(X_test.shape)
print(Y_test.shape)

In the following cells transfer-learning is performed from the pre-trained model MobileNet v2

In [ ]:
# initialize the initial learning rate, number of epochs to train for, batch size
INIT_LR = 1e-4
EPOCHS = 50
BS = 32

In [ ]:
# load the MobileNetV2 network, pretrained on ImageNet dataset
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# build the headModel, that is the part on top of the model 
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)

# connect the head part to the base model defined before in order to obtain the final model to perform binary classification
model = Model(inputs=baseModel.input, outputs=headModel)

# freeze all the layers of the base model so they will not be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# compile our model
print("Compiling the model ...")
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the head of the network
print("Training the head ofthe model, please wait ...")
H = model.fit(
	X_train, Y_train, BS,
	steps_per_epoch=len(X_train) // BS,
	validation_data=(X_test, Y_test),
	validation_steps=len(X_test) // BS,
	epochs=EPOCHS)
print("Training completed")

In the following cells the trained model is saved in order to be exproted in the C++ source code for testing it

In [ ]:
model.save('/--folder in which you want to save the model path ----/model', save_format = 'tf')

In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

loaded = tf.saved_model.load('/--folder in which you want to save the model path ----/model')
infer = loaded.signatures['serving_default']
f = tf.function(infer).get_concrete_function(input_1=tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32))
f2 = convert_variables_to_constants_v2(f)
graph_def = f2.graph.as_graph_def()

with tf.io.gfile.GFile('/--folder in which you want to save the model path ----/model/model.pb', 'wb') as f:
    f.write(graph_def.SerializeToString())